In [57]:
import os

import h5py
import matplotlib.pyplot as plt
import meshio
import numpy as np
import warnings

from dolfinx import cpp, fem, io, mesh, nls, plot
from mpi4py import MPI
from petsc4py import PETSc

import commons, geometry, grapher
warnings.simplefilter('ignore')

### Irregular Surface

In [12]:
img = np.asarray(plt.imread('data/current_constriction/test11.tif')[:, :, 0], dtype=np.uint8)

In [38]:
coords = np.asarray(np.argwhere(img == 1), dtype=np.int32)

In [52]:
def mesh_surface(coords, xmax=470, ymax=470):
    points = {}
    new_points = {}
    count = 0
    for row in coords:
        points[(row[0], row[1])] = count
        count += 1
    points_set = set(points.keys())
    no_full_neighbors = set()
    triangles = []
    for (x0, y0) in points_set:
        p0 = points[(x0, y0)]
        neighbors = [
            (int(x0 + 1), y0),
            (int(x0 + 1), int(y0 + 1)),
            (x0, int(y0 + 1))
        ]
        neighbor_points = [p0]
        for p in neighbors:
            v = points.get(p)
            neighbor_points.append(v)

        midpoint = (x0 + 0.5, y0 + 0.5)
        if midpoint[0] > xmax or midpoint[1] > ymax:
            continue
        points[midpoint] = count
        p2 = count
        count += 1
        for i in range(4):
            p0 = neighbor_points[i]
            if i == 3:
                p1 = neighbor_points[0]
            else:
                p1 = neighbor_points[i + 1]
            if not p0 is None and not p1 is None:
                triangles.append(
                    (p0, p1, p2)
                )
                

    return triangles, points

In [53]:
triangles, points = mesh_surface(coords)

In [58]:
cell_types = commons.CellTypes()
cell_type = cell_types.triangle
cells = np.asarray(triangles, dtype=np.int32)
max_row_idx = max(points.values())
new_points = np.zeros((max_row_idx +1, 2))
print(new_points.shape)
for k, v in points.items():
    new_points[v, :] = k
cell_data = np.zeros((max_row_idx +1, ), dtype=np.int32)
out_mesh = meshio.Mesh(points=new_points, cells={cell_type: cells}, cell_data={"name_to_read": [cell_data]})

(155249, 2)


ValueError: Incompatible cell data. Cell block 0 ('triangle') has length 298271, but corresponding cell data item has length 155249.

In [22]:
np.argwhere(x>1), x.shape

(array([[0, 2],
        [1, 0],
        [1, 1],
        [1, 2]]),
 (2, 3))

In [2]:
outdir = 'mesh/study_4/26-26-7.25/0.5'
outdir = 'mesh/1/051-051-051_000-000-000'
tria_meshname = f'{outdir}/tria.xdmf'
tetr_meshname = f'{outdir}/tetr.xdmf'

with io.XDMFFile(MPI.COMM_WORLD, f"{tetr_meshname}", "r") as infile3:
    domain = infile3.read_mesh(cpp.mesh.GhostMode.none, 'Grid')
    ct = infile3.read_meshtags(domain, name="Grid")
domain.topology.create_connectivity(domain.topology.dim, domain.topology.dim - 1)
# with io.XDMFFile(MPI.COMM_WORLD, f"{tria_meshname}", "r") as infile3:
#     surf = infile3.read_mesh(cpp.mesh.GhostMode.none, 'Grid')
#     ft = infile3.read_meshtags(domain, name="Grid")

In [4]:
surfaces = mesh.locate_entities_boundary(domain, 2, lambda x: np.isreal(x[1]))

In [5]:
news = mesh.locate_entities_boundary(domain, 0, lambda x: np.isreal(x[1]))

In [6]:
(surfaces.shape, news.shape, mesh.exterior_facet_indices(domain.topology).shape)

((1517852,), (955465,), (1517852,))

In [ ]:
domain.find

In [7]:
data3 = h5py.File(tetr_meshname.replace(".xdmf", ".h5"), "r")
# data2 = h5py.File(tria_meshname.replace(".xdmf", ".h5"), "r")

In [46]:
data2.keys(), data3.keys()

(<KeysViewHDF5 ['data0', 'data1', 'data2']>,
 <KeysViewHDF5 ['data0', 'data1', 'data2']>)

In [43]:
data2['data0'].shape, data3['data0'].shape

((565884, 3), (565884, 3))

In [44]:
data2['data1'].shape, data3['data1'].shape

((739098, 3), (2248586, 4))

In [45]:
data2['data2'].shape, data3['data2'].shape 

((739098,), (2248586,))

In [8]:
np.max(surfaces)

11875687

In [9]:
np.max(news)

962544

In [10]:
data3.keys()

<KeysViewHDF5 ['data0', 'data1', 'data2']>

In [11]:
data3['data0'].shape, data3['data1'].shape, data3['data2'].shape

((962545, 3), (5558381, 4), (5558381,))

In [15]:
data3['data1'][0, :]

array([121901, 121906, 121907, 121904])

(1517852,)